## Simply average long period analysis
---

In [85]:
import numpy as np
import pandas as pd
from scripts import StockPriceDatabase

report = pd.read_csv("../database/reportData/Annual_Value.csv", index_col=[0])
report["Date"] = report["Date"].astype('datetime64[s]')
spd = StockPriceDatabase("../database/stockPrice/")
wig = pd.read_csv("../database/designData/WIGs.csv")

In [101]:
M_DAY_DISTANCE = 30

In [163]:
differential_value = pd.DataFrame()

for _, row in report.iterrows():
    date, ticker = row["Date"], row["Ticker"]
    # Company
    stock_price = spd.get_stock_price(row["Ticker"])
    stock_price.index = stock_price.index.astype('datetime64[s]')
    close_avg_earlier = stock_price[np.all([stock_price.index > row["Date"] + np.timedelta64(-2*M_DAY_DISTANCE, 'D'),
                                            stock_price.index < row["Date"] + np.timedelta64(-1*M_DAY_DISTANCE, 'D')], axis=0)]["Close"].mean()
    close_avg_after   = stock_price[np.all([stock_price.index > row["Date"] + np.timedelta64(1*M_DAY_DISTANCE, 'D'),
                                            stock_price.index < row["Date"] + np.timedelta64(2*M_DAY_DISTANCE, 'D')], axis=0)]["Close"].mean()
    company_growth_index = (close_avg_after - close_avg_earlier) / close_avg_earlier * 100
    
    # WIGs
    try:
        wig_ticker = np.where(wig.to_numpy() == "WIG_POLAND")[1][0]
    except:
        break
    stock_price = spd.get_stock_price(wig.columns[wig_ticker])
    stock_price.index = stock_price.index.astype('datetime64[s]')
    close_avg_earlier = stock_price[np.all([stock_price.index > row["Date"] + np.timedelta64(-2*M_DAY_DISTANCE, 'D'),
                                            stock_price.index < row["Date"] + np.timedelta64(-1*M_DAY_DISTANCE, 'D')], axis=0)]["Close"].mean()
    close_avg_after   = stock_price[np.all([stock_price.index > row["Date"] + np.timedelta64(1*M_DAY_DISTANCE, 'D'),
                                            stock_price.index < row["Date"] + np.timedelta64(2*M_DAY_DISTANCE, 'D')], axis=0)]["Close"].mean()
    wig_growth_index = (close_avg_after - close_avg_earlier) / close_avg_earlier * 100


    

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals